# Multivariate Limit Theorems via ACSV

This worksheet implements SageMath code for automatically computing central limit theorems from multivariate rational generating functions. Details on the theory behind the package can be found in the paper [*Central Limit Theorems via Analytic Combinatorics in Several Variables*](https://arxiv.org/abs/2211.15492) by Stephen Melczer and Tiadora Ruza. 

Our code relies on the `sage_acsv` package available from the git repository [https://github.com/ACSVmath/sage_acsv](https://github.com/ACSVmath/sage_acsv). Our code for central limit theorems will be incorporated into future versions of the core `sage_acsv` package. The last stable version of `sage_acsv` can be installed from the command line on a computer with SageMath installed by running

`sage -pip install sage-acsv`

**Note:** The stable version installed by pip does not currently support irrational direction vectors. Because of this, the current code does not *prove* central limit theorems with irrational peaks by default (it returns a warning and what the limit theorem would be if the required conditions could be proven). If you want to prove central limit theorems for such cases you must download and manually install the code from the branch [https://github.com/ACSVMath/sage_acsv/tree/irrational_directions](https://github.com/ACSVMath/sage_acsv/tree/irrational_directions) and pass the argument `support_irrat=True` to the `getLCLT` function.

## Code for the Package

In [1]:
# Import the sage_acsv package for verifying minimal critical points
from sage_acsv import *

In [2]:
# Helper function
def HessianMatrixWithLog(H, vs, r):
    r"""Computes the Hessian matrix appearing in LCLT.

    INPUT:

    * ``H`` -- a polynomail (the denominator of the rational GF `F`)
    * ``vs`` -- list of variables ``z_1, ..., z_d``
    * ``r`` -- direction vector of length `d` with positive entries

    OUTPUT:

    The Hessian matrix with rational function entries in the variables of ``vs``.
    """
    z_d = vs[-1]
    d = len(vs)

    # Build d x d matrix of U[i,j] = z_i * z_j * H'_{z_i * z_j}
    U = matrix(
        [
            [
                v1 * v2 * H.derivative(v1, v2)/(z_d * H.derivative(z_d))
                for v2 in vs
            ] for v1 in vs
        ]
    )
    V = [(r[k] / r[-1]) for k in range(d)]

    # Build (d-1) x (d-1) Matrix for Hessian
    Hess = [
        [
            V[i] * V[j] + U[i][j] - V[j] * U[i][-1] - V[i]*U[j][-1]
            + V[i] * V[j] * U[-1][-1]
            for j in range(d-1)
        ] for i in range(d-1)
    ]
    for i in range(d-1):
        Hess[i][i] = Hess[i][i] + V[i]

    # Return Hessian
    return matrix(Hess)

In [3]:
# Main function to compute LCLT
def getLCLT(F, main_var, as_symbolic=False, support_irrat=False):
    r"""Take a multivariate rational generating function, check if it admits a 
    minimal critical point of a form implying a local central limit theorem, and
    (if so) return the local central limit theorem.

    INPUT:

    * ``F`` -- The rational function ``G/H`` in ``d`` variables. This function is
      assumed to have a combinatorial expansion.
    * ``main_var`` -- The variable that marks the ``size`` of the objects (so that the limit
    theorem holds as the exponent of ``main_var`` goes to infinity).
    * ``as_symbolic`` -- If ``True``, returns the limit theorem as an expression from the symbolic
        ring ``SR`` in the variable ``n``. If ``False``, the default, returns a tuple 
        (a, n^b, pi^b, C, D, v) such that the local central limit theorem is specified by the
        function f(s) = a^n * n^b * pi^b * C * exp(-((s-n*v)*D*(s-n*v).transpose())/2/n)

    OUTPUT:

    A representation of the local central limit theorem, either as a list of tuples,
    or as a symbolic expression.
    """

    # Initialize quantities
    G, H = F.numerator(), F.denominator()
    zvariables = [v for v in H.variables() if v != main_var]
    R = PolynomialRing(QQ,zvariables + [main_var])
    vs = R.gens()
    
    # Make sure G and H are coprime, and that H does not vanish at 0
    G, H = RationalFunctionReduce(G, H)
    G, H = R(G), R(H)
    if H.subs({v: 0 for v in vs}) == 0:
        raise ValueError("Denominator vanishes at 0.")

    # Find rho
    P = H.subs({v: 1 for v in vs[0:-1]})
    rts = [r for r in QQ[vs[-1]](P).roots(AA, multiplicities=False) if r>0]
    if len(rts) == 0:
        raise ValueError("H(1,rho)=0 has no positive solution.")
    rho = min(rts)
    sbs = {v: 1 for v in vs[0:-1]} | {vs[-1]:rho}

    # Check numerator and denominator requirements are met
    if (H.derivative(vs[-1])).subs(sbs) == 0:
        raise ValueError("The partial derivative of the denominator at (1, rho) is 0.")
    
    if G.subs(sbs) == 0:
        raise ValueError("The numerator at (1, rho) is 0.")

    # Get direction for LCLT
    m = [H.derivative(v).subs(sbs)/(rho*H.derivative(vs[-1]).subs(sbs)) for v in vs[0:-1]] + [1]

    # Check if we have a rational direction, otherwise print warning unless user specifies custom sage_acsv branch
    isRational = True
    if max([k.degree() for k in m[0:-1]]) == 1:
        multiple = LCM([QQ(k).denom() for k in m])
        r = [ZZ(multiple*k) for k in m]
    else:
        if not support_irrat:
            acsv_logger.warning(
                    "Direction is not rational. If you have a version of sage_acsv that supports "
                    f"irrational directions, call the same function with irrat_check = False to try and prove an LCLT." 
                    f" Assuming that we have a strictly minimal critical point of the required form, the following LCLT holds."
            )
            isRational = False
        r = m

    RR, (q, lambda_, u_) = PolynomialRing(QQ, 'q, lambda_, u_').objgens()
    expanded_R, _ = PolynomialRing(QQ, len(vs)+3, vs + (q, lambda_, u_)).objgens()

    vs = [expanded_R(v) for v in vs]
    q, lambda_, u_ = expanded_R(q), expanded_R(lambda_), expanded_R(u_)
    vsT = vs + [q, lambda_]
    G, H = expanded_R(G), expanded_R(H)

    all_variables = (vs, lambda_, q, u_)
    d = len(vs)
    rd = r[-1]

    if isRational:
        for _ in range(MAX_MIN_CRIT_RETRIES):
            try:
                # Find minimal critical points in Kronecker Representation
                min_crit_pts = MinimalCriticalCombinatorial(
                    G, H, all_variables,
                    r=r
                )

                if len(min_crit_pts) != 1 or min_crit_pts != [[1 for v in range(d-1)] + [rho]]:
                    raise ValueError("The point (1,rho) is not the only critical point with this coordinate-wise modulus.")
                break
            except Exception as e:
                # In case form doesn't separate, we want to try again
                if isinstance(e, ACSVException) and e.retry:
                    acsv_logger.warning(
                        "Randomly generated linear form was not suitable, "
                        f"encountered error: {e}\nRetrying..."
                    )
                    continue
                elif support_irrat:
                    acsv_logger.warning(
                        "Error occurred during check of minimal critical point. "
                        f"This is preliminary work in progress. Please ensure that "
                        f"your version of sage_acsv supports irrational directions. "
                        f"If it doesn't, call this function again with support_irrat = False "
                        f"to see potential results. "
                        f"Encountered error: {e}\n"
                    )
                    return
                else:
                    raise e
    
    sbs = {v:1 for v in vsT[0:-3]} | {vsT[-3]:rho}
    Hess = HessianMatrixWithLog(H, vsT[0:-2], r)
    Hess = Hess.subs({v:1 for v in Hess.base_ring().gens()[0:-4]} | {Hess.base_ring().gens()[-4]:rho})
    Det = Hess.determinant()

    if Det == 0:
        raise ValueError("Hessian determinant is 0.")

    # Values appearing in asymptotics
    base = 1/rho
    constant = - AA(G.subs(sbs) / rho / H.derivative(vs[-1]).subs(sbs) / (2^(d-1) * Det).sqrt())
    exponent = (1-d)/2

    s = matrix((var('s', n=d-1)))
    invHess = Hess.inverse()
    
    n = SR.var('n')
    result = (base, n^exponent, pi^exponent, constant, invHess, matrix(m[:-1]))
    
    if as_symbolic:
        (a, b, c, d, e, f) = result
        
        if a.degree() <= 2:
            a = base.radical_expression()
        if d.degree() <= 2:
            d = constant.radical_expression()
            
        if max([AA(k).degree() for k in e.list()] + [AA(k).degree() for k in m[:-1]]) <=2:
            sfactor = exp(-(((s-n*f)*e*(s-n*f).transpose())[0,0]).simplify()/2/n)
        else: 
            sfactor = exp(-(((s-n*f)*e*(s-n*f).transpose())[0,0])/2/n)
        
        result = a^n * b * c * d * sfactor

    return result

## Examples Using the Package

#### Strings with Tracked Letters

In [5]:
# Limit theorem for number s0 of 0s in binary strings of length n
var('z t')
show(getLCLT(1/(1-z*t-t), t, as_symbolic = True))

sqrt(2)*2^n*e^(-1/2*(n - 2*s0)^2/n)/(sqrt(pi)*sqrt(n))

In [8]:
# Limit theorem for number s0 of 0s in 0123-strings of length n
var('z t')
show(getLCLT(1/(1-z*t-3*t), t, as_symbolic = True))

2*sqrt(2/3)*4^n*e^(-1/6*(n - 4*s0)^2/n)/(sqrt(pi)*sqrt(n))

In [7]:
# Limit theorem for number s0 of 0s and s1 of 1s in 012-strings of length n
var('x y t')
F = 1/(1-x*t-y*t-t)
show(getLCLT(F, t, as_symbolic = True))

3/2*sqrt(3)*3^n*e^(-1/2*((n - 2*s0 - s1)*(n - 3*s0) + (n - s0 - 2*s1)*(n - 3*s1))/n)/(pi*n)

In [11]:
# Limit theorem for number s0 of 0s and s1 of 1s in 01234-strings of length n
var('x y t')
F = 1/(1-x*t-y*t-3*t)
show(getLCLT(F, t, as_symbolic = True))

5/2*sqrt(5/3)*5^n*e^(-1/6*((n - 4*s0 - s1)*(n - 5*s0) + (n - s0 - 4*s1)*(n - 5*s1))/n)/(pi*n)

#### Compositions with Tracked Summands

In [12]:
# Limit theorem for number s0 of the 1s in compositions of length n
var('x t')
F = 1/(1-x*t-t^2/(1-t))
show(getLCLT(F, t, as_symbolic = True))

sqrt(2/5)*2^n*e^(-1/10*(n - 4*s0)^2/n)/(sqrt(pi)*sqrt(n))

In [13]:
# Limit theorem for number s0 of 1s and s1 of 2s in compositions of length n
var('x y t')
F = 1/(1-x*t-y*t^2-t^3/(1-t))
show(getLCLT(F, t, as_symbolic = True))

8*sqrt(1/35)*2^n*e^(-1/70*(7*(n - 4*s0)^2 + 5*(n - 8*s1)^2)/n)/(pi*n)

In [14]:
# Limit theorem for number s0 of 1s, s1 of 2s and s2 of 3s in compositions of length n
var('z1 z2 z3 t')
F = 1/(1-z1*t-z2*t^2-z3*t^3-t^4/(1-t))
show(getLCLT(F, t, as_symbolic = True))

32*sqrt(2/107)*2^n*e^(-1/214*((24*n - 87*s0 - 4*s1 - 28*s2)*(n - 4*s0) + (19*n - 2*s0 - 128*s1 - 40*s2)*(n - 8*s1) + (13*n - 7*s0 - 20*s1 - 140*s2)*(n - 16*s2))/n)/(pi^(3/2)*n^(3/2))

#### Permutations with Restricted Cycles

In [21]:
# We can run examples with irrational directions but by default we don't check the assumptions for LCLT
var('z t')
F = 1/(1-t-z*t^2)
show(getLCLT(F, t, as_symbolic = True))

1.710862642974252?*(1/2*sqrt(5) + 1/2)^n*e^(1/8*(n*(sqrt(5) - 1) - 2*sqrt(5)*s0)*(n*(sqrt(5) - 5) + 10*s0)/n)/(sqrt(pi)*sqrt(n))

In [22]:
# If a newer branch of sage_acsv that supports irrational directions is installed, we can prove the result
# by specifying the parameter support_irrat = True
var('z t')
F = 1/(1-t-z*t^2)
show(getLCLT(F, t, as_symbolic = True, support_irrat = True))

1.710862642974252?*(1/2*sqrt(5) + 1/2)^n*e^(1/8*(n*(sqrt(5) - 1) - 2*sqrt(5)*s0)*(n*(sqrt(5) - 5) + 10*s0)/n)/(sqrt(pi)*sqrt(n))

In [4]:
var('x y t')
F = 1/(1-t-x*t^2-y*t^3)
show(getLCLT(F, t, as_symbolic=True, support_irrat = True))

5.030774639167280?*1.839286755214161?^n*e^(-1/2*((-0.1828035329682955?*n + s0)*(-4.331318273807052?*n + 15.75001457299826?*s0 + 14.61097905351492?*s1) + (-0.0993882723561560?*n + s1)*(-5.688465294663696?*n + 14.61097905351492?*s0 + 30.36099362651318?*s1))/n)/(pi*n)

In [8]:
# Trying to prove an LCLT with an irrational direction when a newer branch is not installed gives the following error
var('x y t')
F = 1/(1-t-x*t^2-y*t^3)
show(getLCLT(F, t, as_symbolic=True, support_irrat = True))

None